In [11]:
import boto3
import json

### `Lambda`

In [12]:
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# TODO: open the zip file containing your Lambda function
with open('pi_sim.zip', 'rb') as f:
    lambda_zip = f.read()

In [13]:
try:
    # If function hasn't yet been created, create it
    # TODO: replace the placeholders with your function specifics
    response = aws_lambda.create_function(
        FunctionName='pi_sim',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='lambda_funtion.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        FunctionName='pi_sim',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
lambda_arn

'arn:aws:lambda:us-east-1:654654411374:function:pi_sim'

In [14]:
# can invoke once serially, or invoke in parallel via Step functions (below)
r = aws_lambda.invoke(FunctionName='pi_sim',
                      InvocationType='RequestResponse',
                      Payload=json.dumps({'num_points': 200})
                      )
json.loads(r['Payload'].read())

{'statusCode': 200, 'body': {'pi_estimate': 3.22, 'num_points': 200}}

### `Step` Functions

In [15]:
sfn = boto3.client('stepfunctions')

def make_def(lambda_arn):
    definition = {
      "Comment": "My State Machine",
      "StartAt": "Map",
      "States": {
        "Map": {
          "Type": "Map",
          "End": True,
          "Iterator": {
            "StartAt": "Lambda Invoke",
            "States": {
              "Lambda Invoke": {
                "Type": "Task",
                "Resource": "arn:aws:states:::lambda:invoke",
                "OutputPath": "$.Payload",
                "Parameters": {
                  "Payload.$": "$",
                  "FunctionName": lambda_arn
                },
                "Retry": [
                  {
                    "ErrorEquals": [
                      "Lambda.ServiceException",
                      "Lambda.AWSLambdaException",
                      "Lambda.SdkClientException",
                      "Lambda.TooManyRequestsException",
                      "States.TaskFailed"
                    ],
                    "IntervalSeconds": 2,
                    "MaxAttempts": 6,
                    "BackoffRate": 2
                  }
                ],
                "End": True
              }
            }
          }
        }
      }
    }
    return definition

In [16]:
sf_def = make_def(lambda_arn)

try:
    response = sfn.create_state_machine(
        name='monte_carlo_sm',
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn'],
        type='EXPRESS'
    )
except sfn.exceptions.StateMachineAlreadyExists:
    response = sfn.list_state_machines()
    state_machine_arn = [sm['stateMachineArn'] 
                         for sm in response['stateMachines'] 
                         if sm['name'] == 'monte_carlo_sm'][0]
    response = sfn.update_state_machine(
        stateMachineArn=state_machine_arn,
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn']
    )

In [17]:
response = sfn.list_state_machines()
response

{'stateMachines': [{'stateMachineArn': 'arn:aws:states:us-east-1:654654411374:stateMachine:hello_world_sm',
   'name': 'hello_world_sm',
   'type': 'EXPRESS',
   'creationDate': datetime.datetime(2025, 4, 23, 14, 53, 26, 623000, tzinfo=tzlocal())},
  {'stateMachineArn': 'arn:aws:states:us-east-1:654654411374:stateMachine:monte_carlo_sm',
   'name': 'monte_carlo_sm',
   'type': 'EXPRESS',
   'creationDate': datetime.datetime(2025, 4, 23, 17, 17, 22, 99000, tzinfo=tzlocal())},
  {'stateMachineArn': 'arn:aws:states:us-east-1:654654411374:stateMachine:word_count_sm',
   'name': 'word_count_sm',
   'type': 'EXPRESS',
   'creationDate': datetime.datetime(2025, 4, 18, 14, 25, 48, 340000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '815f3133-0c11-42f3-89a6-08264d52f46b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '815f3133-0c11-42f3-89a6-08264d52f46b',
   'date': 'Wed, 23 Apr 2025 22:17:36 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length

In [26]:
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'monte_carlo_sm'][0]

# Do not invoke more than 10 workers at a time or you risk your AWS Academy account being deactivated
# here, we (conservatively) only concurrently invoke lambda function 5 times
data = [{"num_points": 20000} for i in range(8)]
data # note 5 distinct tasks (mapping onto 5 distinct Lambda invocations)

[{'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000},
 {'num_points': 20000}]

In [27]:
sfn_response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='sync_test',
    input=json.dumps(data)
)

print(sfn_response['output'])

[{"statusCode":200,"body":{"pi_estimate":3.1378,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1418,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.155,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1276,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1424,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1624,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1344,"num_points":20000}},{"statusCode":200,"body":{"pi_estimate":3.1526,"num_points":20000}}]


In [28]:
pi_sum, counter = 0, 0

for thread in json.loads(sfn_response['output']):
    pi_sum += thread['body']['pi_estimate']
    counter += 1

print(pi_sum/counter)

3.14425
